# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

Calculation started (calculation_id=8cc7177b-67e0-05d4-771f-f2c4439cb4a8) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
<SparkContext master=athena appName=default>



<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=5ac7177b-6d92-5164-298b-d9631767a0a5) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp, lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

Calculation started (calculation_id=a6c7177b-72b0-bb35-0c01-9c54406049ee) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [4]:
#export
def user():
    # Returns a string consisting of your GT username.
    return 'rzhao97'

Calculation started (calculation_id=0ac7177b-77db-912c-4ce0-8b32b5a05b3c) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [5]:
#export
def long_trips(trips):
    # Returns a Dataframe (trips) with Schema the same as :trips:
    df = trips.filter(trips.trip_distance >= 2)
    return df

Calculation started (calculation_id=3ac7177b-7952-416f-32cd-1807825f743a) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [6]:
#export
def manhattan_trips(trips, lookup):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount
    lookup = lookup.withColumnRenamed('LocationID', 'DOLocationID')
    df = trips.join(lookup, 'DOLocationID', 'outer')
    df = df.filter(df.Borough == 'Manhattan')
    
    df = df.groupBy('DOLocationID').agg({'passenger_count': 'sum'})
    df = df.select(col('DOLocationID'),
                  col('sum(passenger_count)').alias('pcount'))
    
    df = df.sort(df['pcount'].desc()).limit(20)
    return df

Calculation started (calculation_id=bac7177b-7aa0-6eac-980c-f045b6c172b3) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [40]:
#export
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe (wp) with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    filtered = trips.join(mtrips, 'DOLocationID', 'inner')
    
    raw = trips.groupBy('PULocationID').agg({'PULocationID': 'count', 'total_amount': 'average'})
    raw = raw.withColumnRenamed('count(PULocationID)', 'total')
    filtered = filtered.groupBy('PULocationID').agg({'PULocationID': 'count'})
    filtered = filtered.withColumnRenamed('count(PULocationID)', 'popular')
    
    df = raw.join(filtered, 'PULocationID', 'inner')
    df = df.withColumnRenamed('avg(total_amount)', 'amount')
    
    df = df\
        .withColumn("weighted_profit", 
                    (col('amount') * col("popular") / col("total")).cast('decimal(24, 10)'))\
        .drop('amount', 'total', 'popular')
    
    #df = df.sort(df['weighted_profit'].desc()).limit(20)

    return df

Calculation started (calculation_id=46c71789-b085-1f16-2610-a1b0602d43c3) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [31]:
#export
def final_output(wp, lookup): 
    # Returns a Dataframe (final) with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    wp = wp.withColumnRenamed('PULocationID', 'LocationID')
    df = wp.join(lookup, 'LocationID', 'inner')
    
    df = df.select(col('Zone'),
                  col('Borough'),
                  col('weighted_profit'))
    
    df = df.sort(df['weighted_profit'].desc()).limit(20)
    
    return df

Calculation started (calculation_id=b8c71780-f401-546f-f9ea-8367d1236a64) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading.
</div>

In [35]:
print(user())
trips, lookup = load_data()
#trips = long_trips(trips)

Calculation started (calculation_id=acc71788-d07d-7ef7-ae2c-24c28592f809) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
rzhao97
Trip Count:  7667792



In [36]:
trips.show()
lookup.show()
trips = long_trips(trips)

Calculation started (calculation_id=32c71789-1283-79aa-39de-95ec9836d67e) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|              1|          1.5|         1|                 N|         151|         239|           1|        7.0|  0.5|    0.5|      1.65|         0.

In [37]:
mtrips = manhattan_trips(trips, lookup)
mtrips.show()

Calculation started (calculation_id=0ac71789-2822-85dd-ae85-0f46a1a35073) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+------------+------+
|DOLocationID|pcount|
+------------+------+
|         236|142135|
|         161|113973|
|         230|109837|
|         239|100645|
|         238| 97914|
|         162| 97622|
|          48| 96469|
|         170| 93111|
|         231| 93056|
|         263| 92496|
|          79| 91976|
|         142| 91225|
|         141| 87554|
|         237| 87042|
|         140| 82199|
|          68| 77125|
|         234| 72878|
|         186| 71638|
|         163| 70748|
|          13| 70077|
+------------+------+



In [41]:
wprof = weighted_profit(trips, mtrips)
wprof.show()

Calculation started (calculation_id=bcc71789-c88e-4b28-0c2c-8fa24120cd6b) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+------------+---------------+
|PULocationID|weighted_profit|
+------------+---------------+
|           6|  31.7844444444|
|          99|  29.3425000000|
|          93|  24.4761070519|
|         187|  24.3577777778|
|          10|  23.6426704059|
|         156|  23.3555029586|
|         194|  23.1245775151|
|         215|  22.1741149394|
|         206|  21.4918934911|
|           5|  20.2962500000|
|           8|  19.9653600000|
|         138|  17.9367798201|
|         171|  17.7970609810|
|         132|  16.9628723824|
|         130|  14.8798351888|
|         105|  14.7962500000|
|          58|  14.6131952663|
|         219|  14.3371752194|
|         157|  13.9404371607|
|          28|  13.3594638254|
+------------+---------------+



In [42]:
final_output(wprof, lookup).show()

Calculation started (calculation_id=8cc7178a-026b-2fa5-32cf-0c8c4903af28) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|       Arden Heights|Staten Island|  20.2962500000|
|Arrochar/Fort Wad...|Staten Island|  31.7844444444|
|        Astoria Park|       Queens|  19.9653600000|
|        Baisley Park|       Queens|  23.6426704059|
|Briarwood/Jamaica...|       Queens|  13.3594638254|
|        Country Club|        Bronx|  14.6131952663|
|Flushing Meadows-...|       Queens|  24.4761070519|
|     Freshkills Park|Staten Island|  29.3425000000|
|Governor's Island...|    Manhattan|  14.7962500000|
|             Jamaica|       Queens|  14.8798351888|
|         JFK Airport|       Queens|  16.9628723824|
|   LaGuardia Airport|       Queens|  17.9367798201|
|     Mariners Harbor|Staten Island|  23.3555029586|
|             Maspeth|       Queens|  13.9404371607|
|  Murray Hill-Queens|       Queens|  17.7970609810|
|       Port Richmond|S

In [43]:
#main("large", "s3://cse6242-rzhao97/testing0")

Calculation started (calculation_id=48c7178a-4b4e-6a41-63ed-b2b2e99bebce) in (session=52c71779-fa3d-1aab-7d3c-b18157351b2b). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
rzhao97
Trip Count:  187203269
+--------------------+-------------+---------------+
|                Zone|      Borough|weighted_profit|
+--------------------+-------------+---------------+
|         Jamaica Bay|       Queens|  16.2591730587|
|        Astoria Park|       Queens|  26.2972814509|
|        Baisley Park|       Queens|  31.2229521004|
|        Battery Park|    Manhattan|  14.5730134009|
|Briarwood/Jamaica...|       Queens|  22.8642898034|
|              Corona|       Queens|  20.8005378133|
|Eltingville/Annad...|Staten Island|  16.5878977273|
|Flushing Meadows-...|       Queens|  29.5027711258|
|             Jamaica|       Queens|  21.9765709668|
|         JFK Airport|       Queens|  17.7158076177|
|   LaGuardia Airport|       Queens|  19.6668371981|
|             Maspeth|       Queens|  20.1989088562|
|  Murray Hill-Queens|       Queens|  14.3183800323|
|     Randalls Island|    Manhattan|  26.6153319220|
|Saint Michaels Ce...|       Queens|  14.4709